# Neural Network from sklearn
Conny Lin | June 15, 2020 | updated June 18, 2020

[sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)


In [6]:
# import local libraries using host specific paths
import socket, sys
hostname = socket.gethostname().split('.')[0]
# set local path settings based on computer host
if hostname == 'PFC':
    pylibrary = '/Users/connylin/Dropbox/Code/proj'
elif hostname == 'Angular-Gyrus':
    pylibrary = '/Users/connylin/Code/proj'
else:
    assert False, 'host computer not regonized'
# load local libraries
if pylibrary not in sys.path:
    sys.path.insert(1, pylibrary)
# import brainstation_capstone
# import other standard paths and local variables
from brainstation_capstone.system import host_paths
localpaths = host_paths.get(hostname)
datapath = localpaths['datapath']

# import standard libraries
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt# import local libraries using host specific paths

getting host computer specific paths


Try a simple NN from sklearn. But usually won't use this package. Normally use packages like *Keras*, *TensorFlow*, *PyTorch*, etc., which are all specifically designed for neural networks.

In [8]:
def PlotBoundaries(model, X, Y) :
    '''
    Helper function that plots the decision boundaries of a model and data (X,Y)
    code modified from: https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html
    '''
    
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1,X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.4)

    #Plot
    plt.scatter(X[:, 0], X[:, 1], c=Y,s=20, edgecolor='k')
    plt.show()

## Try simple NN from sklearn

In [7]:
# try simple NN
from sklearn.neural_network import MLPClassifier
# import data
from brainstation_capstone.etl.datatransform import Nutcracker
X, X_test, y, y_test = Nutcracker(datapath).mldata(feature_reduction='None')

None
None


In [13]:
NN_model = MLPClassifier(hidden_layer_sizes=(1),solver='lbfgs')
NN_model.fit(X_train, y_train);

In [15]:
print(f"Train Score: {NN_model.score(X_train,y_train): 0.3f}")
print(f"Test Score: {NN_model.score(X_test,y_test): 0.3f}")

Train Score:  0.869
Test Score:  0.869


solver lbfgs and layer size =1 gives similar results as other ML models.

In [12]:
import timeit
from sklearn.neural_network import MLPClassifier
# import data
from brainstation_capstone.etl.datatransform import Nutcracker
X, X_test, y, y_test = Nutcracker(datapath).mldata(feature_reduction='None')
# set up time
start_time = timeit.timeit()
print('timer start')
print(f'test timer {(timeit.timeit() - start_time)/60:.2f} min')
# start model
NN_model = MLPClassifier()
NN_model.fit(X, y)
# print results
print(f"Train Score: {NN_model.score(X, y): 0.3f}")
print(f"Test Score: {NN_model.score(X_test, y_test): 0.3f}")
# print time elapsed
print(f'took {(timeit.timeit() - start_time)/60:.2f} min')

counter start
test timer -0.00 min
Train Score:  0.429
Test Score:  0.429


TypeError: unsupported operand type(s) for -: 'function' and 'float'

untuned model gives 91% results. highest of all ML models

find out what hyperparamters the untuned model used.

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

* hidden_layer_sizes: 100
* activation: ’relu
* solver: ’adam’
* alpha: 0.0001
* batch_size: ’auto’
* learning_rate: constant
* learning_rate_init: 0.001
* power_t: 0.5
* max_iter: 200
* shuffle: True
* random_state: None
* tol: 1e-4
* verbose: False
* warm_start: False
* momentum: 0.9
* nesterov_momentum: True
* early_stopping: False
* validation_fraction: 0.1
* beta_1: 0.9
* beta_2: 0.999
* epsilon: 1e-8
* n_iter_no_change: 10
* max_fun: 15000

Try TensorFlow. This must be run in the deeplearning env.

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [14]:
y_train.shape

(1600000,)

In [15]:
input_size = X_train.shape[1]
output_size = 0
# Size of the hidden dimension
num_hidden = 10

# Make the placeholder variables:
input_X = tf.placeholder(tf.float32, shape=(None, input_size))
input_y = tf.placeholder(tf.float32, shape=(None, output_size))

# Hidden Unit activations
W1 = tf.Variable(tf.random_normal([input_size, num_hidden]))
b1 = tf.Variable(tf.random_normal([num_hidden])) #bias
hidden_output = tf.nn.relu(tf.add(tf.matmul(input_X,W1), b1))

# Output Unit activations (softmax for multi-class)
W2 = tf.Variable(tf.random_normal([num_hidden, output_size]))
b2 = tf.Variable(tf.random_normal([output_size]))
net_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, W2), b2))

# Use cross entropy loss (we're doing classification)
# This is where we link our network to the objective function.
loss = tf.losses.softmax_cross_entropy(input_y, net_output)

# Get accuracy measure
# acc will store the accuracy score of the true labels and our net output
acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(input_y, 1), 
                                  predictions=tf.argmax(net_output, 1))

# Use gradient descent with a learning rate of 0.1
optimizer = tf.train.GradientDescentOptimizer(0.1)

# Objective is minimize the loss
train = optimizer.minimize(loss)

# Create a session for our netowrk:
sess = tf.Session()
init = tf.group(tf.global_variables_initializer(), 
                tf.local_variables_initializer())
sess.run(init)

In [16]:
input_y

<tf.Tensor 'Placeholder_7:0' shape=(?, 0) dtype=float32>

In [17]:
epochs = 10

# Each epoch is one pass through the data
for epoch in range(epochs):
    # Update the network.
    # feed_dict passes in the input for training
    # train linkes the session to the network
    sess.run(train, feed_dict={input_X: X_train, 
                               input_y: y_train})
    
    if epoch % 500 == 0 :
        # Get the training loss
        training_loss = sess.run(loss, 
                                 feed_dict={input_X: X_train, 
                                            input_y: y_train})

        score = sess.run([acc, acc_op], 
                         feed_dict={input_X: X_train, 
                                    input_y: y_train})
        print(f'Epoch: {epoch}, Loss: {training_loss:.3f}, Accuracy: {score[1]:.3f}')

ValueError: Cannot feed value of shape (1600000,) for Tensor 'Placeholder_7:0', which has shape '(?, 0)'